In [1]:
from pyspark.sql import SparkSession
spark = SparkSession \
        .builder \
        .appName("Streaming WordCount") \
        .master("local[3]") \
        .config("spark.streaming.stopGracefullyOnShutdown", "true") \
        .config("spark.sql.shuffle.partitions", 3) \
        .getOrCreate()

In [2]:
spark

In [3]:
# Creamos un flujo de escucha sobre netcat en localhost:9999
lines_df = spark.readStream \
        .format("socket") \
        .option("host", "localhost") \
        .option("port", "9999") \
        .option('includeTimestamp', 'true')\
        .load()

22/05/05 17:56:53 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.


In [4]:
# leemos las lineas y las pasamos a palabras, y para cada palabra, le añadimos su timestamp
from pyspark.sql.functions import explode, split
words_df = lines_df.select(
    explode(split(lines_df.value, ' ')).alias('palabra'),
    lines_df.timestamp)

In [5]:
from pyspark.sql.functions import window
windowedCounts = words_df.groupBy(
    window(words_df.timestamp, "2 minutes", "1 minute"), words_df.palabra
).count().orderBy('window')

In [6]:
word_count_query = windowedCounts.writeStream \
        .format("console") \
        .outputMode("complete") \
        .option('truncate', 'false')\
        .start()

22/05/05 17:57:00 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/p3/9_2bsgf50ps_4hf29pt9bbc80000gn/T/temporary-1d40fad4-b664-4954-ac55-b9e64e1abb83. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
22/05/05 17:57:00 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [ ]:
word_count_query.awaitTermination()

-------------------------------------------
Batch: 0
-------------------------------------------
+------+-------+-----+
|window|palabra|count|
+------+-------+-----+
+------+-------+-----+

-------------------------------------------
Batch: 1
-------------------------------------------
+------------------------------------------+-------+-----+
|window                                    |palabra|count|
+------------------------------------------+-------+-----+
|{2022-05-05 17:56:00, 2022-05-05 17:58:00}|Mundo  |1    |
|{2022-05-05 17:56:00, 2022-05-05 17:58:00}|Hola   |1    |
|{2022-05-05 17:57:00, 2022-05-05 17:59:00}|Hola   |1    |
|{2022-05-05 17:57:00, 2022-05-05 17:59:00}|Mundo  |1    |
+------------------------------------------+-------+-----+

-------------------------------------------
Batch: 2
-------------------------------------------
+------------------------------------------+-------+-----+
|window                                    |palabra|count|
+------------------------

In [ ]:
word_count_query.stop()